In [3]:
import numpy as np
import pandas as pd
import scipy 
from scipy import stats
from scipy.stats import norm
import lmfit
import time

In [8]:
data = pd.read_excel('prices_from_calibration.xlsx', index_col=0)

In [11]:
price_matrix = data.values
m_t = data.values
sk = data.columns.values

In [21]:
temp_data = data.unstack().reset_index()
temp_data.columns = ['K', 'T', 'price']

In [58]:
price_matrix2 = dd - 3

In [ ]:
data = pd.read_excel('prices_from_calibration_2.xlsx', index_col=0)

In [82]:
def generate_data_need(data):
    temp_data = data.unstack().reset_index()
    temp_data.columns = ['K', 'T', 'price']
    return temp_data
    
def bs_price(sigma, S, T, K, r=0.0411):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * T) / sigma / np.sqrt(T)
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
def resid(params, maturity, strike, y_data):
    S0 = 100
    sigma = params['sigma'].value
    bs_prices = bs_price(sigma, S0, maturity, strike)
    return np.square(np.array(bs_prices) - np.array(y_data))

def all_optimize(temp_data):
    result = list()
    params = lmfit.Parameters()
    params.add('sigma', 0.1, min=0, max=1)
    for i in range(len(temp_data)):
        o1 = lmfit.minimize(resid, params, args=(temp_data['T'].values[i], temp_data['K'].values[i], temp_data['price'].values[i]), method='BFGS')
        result.append(o1.params.valuesdict()['sigma'])
    return result
    
    

    
    
# a = time.time()
# params = lmfit.Parameters()
# params.add('sigma', 0.1, min=0, max=1)

# o1 = lmfit.minimize(resid, params, args=(temp_data['T'].values, temp_data['K'].values, temp_data['price'].values), method='BFGS')
# # print("# Fit using differential_evolution:")
# lmfit.report_fit(o1)
# b = time.time()
# print("Time Elapsed:", b-a)

In [83]:
data2 = pd.read_excel('prices_from_calibration_2.xlsx', index_col=0)
temp_data2 = generate_data_need(data2)

In [87]:
temp_data2.head(5)

,K,T,price
0,80,0.250,21.624420
1,80,0.375,22.493248
2,80,0.500,23.181335
3,80,0.625,23.737136
4,80,0.750,24.392313


In [85]:
implied_vols = all_optimize(temp_data2)

In [102]:
zz = pd.DataFrame(implied_vols, index=pd.MultiIndex.from_arrays([temp_data2['T'].values, temp_data2['K'].values]))

In [113]:
data.T.index

Int64Index([80, 85, 90, 95, 100, 105, 110, 115, 120], dtype='int64')

In [114]:
temp_result = zz.unstack().T.reset_index(drop='level_0')
temp_result.index = data.T.index

In [119]:
temp_result

,0.25,0.375,0.5,0.625,0.75,0.875,1.0,1.125,1.25
80,0.369550,0.350208,0.328635,0.307913,0.299198,0.288520,0.276329,0.268711,0.258496
85,0.351437,0.332045,0.311824,0.292533,0.283154,0.272550,0.261374,0.253761,0.244778
90,0.331549,0.312468,0.293518,0.276143,0.266094,0.256276,0.246061,0.238787,0.230846
95,0.311607,0.292254,0.274908,0.258585,0.248834,0.239651,0.230921,0.224142,0.216967
100,0.290118,0.270928,0.255771,0.240570,0.231538,0.223015,0.215612,0.209562,0.203323
105,0.267628,0.249415,0.235619,0.222119,0.214015,0.206703,0.200403,0.195300,0.189809
110,0.245132,0.228185,0.215678,0.203886,0.196829,0.190884,0.185451,0.181150,0.176883
115,0.225536,0.209786,0.197502,0.186944,0.180865,0.175753,0.171442,0.168053,0.164420
120,0.210160,0.196034,0.183580,0.173261,0.167801,0.163072,0.159164,0.156427,0.153120


In [126]:
from scipy.interpolate import RectBivariateSpline
def data_points_interploate(vs, kind='cubic'):
    
    test_x = np.arange(0.25, 1.30, .05)
    test_y = np.arange(80, 120.5, .5)
    f=RectBivariateSpline(x=vs.index.values, y=vs.columns.values, z=vs.values, kx=3, ky=2)
    data_test = pd.DataFrame(f(test_x, test_y).T, index=test_y, columns=test_x)
    return data_test

In [130]:
result_surface = data_points_interploate(temp_result.T)

In [132]:
result_surface.index -= 100

In [134]:
result_surface.to_pickle('surface.pkl')

In [135]:
ddd = pd.read_pickle('surface.pkl')

In [136]:
ddd

,0.25,0.3,0.35,0.39999999999999997,0.44999999999999996,0.49999999999999994,0.5499999999999999,0.5999999999999999,0.6499999999999999,0.7,...,0.7999999999999998,0.8499999999999999,0.8999999999999999,0.9499999999999998,0.9999999999999998,1.0499999999999998,1.0999999999999999,1.15,1.1999999999999997,1.2499999999999998
-20.0,0.369550,0.362009,0.354219,0.346108,0.337604,0.328635,0.319400,0.311170,0.305416,0.302004,...,0.295544,0.291012,0.285938,0.280822,0.276329,0.272908,0.270100,0.267219,0.263579,0.258496
-19.5,0.367834,0.360248,0.352459,0.344383,0.335935,0.327031,0.317855,0.309666,0.303920,0.300487,...,0.293961,0.289420,0.284372,0.279306,0.274854,0.271437,0.268615,0.265728,0.262121,0.257135
-19.0,0.366097,0.358471,0.350684,0.342641,0.334248,0.325409,0.316296,0.308152,0.302415,0.298958,...,0.292369,0.287824,0.282804,0.277788,0.273375,0.269963,0.267127,0.264238,0.260663,0.255773
-18.5,0.364338,0.356677,0.348893,0.340884,0.332544,0.323770,0.314724,0.306627,0.300901,0.297419,...,0.290770,0.286223,0.281233,0.276266,0.271891,0.268485,0.265638,0.262747,0.259205,0.254407
-18.0,0.362559,0.354867,0.347088,0.339110,0.330823,0.322115,0.313137,0.305092,0.299377,0.295868,...,0.289162,0.284619,0.279659,0.274740,0.270402,0.267002,0.264147,0.261256,0.257747,0.253039
-17.5,0.360758,0.353040,0.345267,0.337320,0.329084,0.320442,0.311536,0.303546,0.297843,0.294306,...,0.287545,0.283009,0.278084,0.273211,0.268909,0.265516,0.262654,0.259764,0.256289,0.251669
-17.0,0.358936,0.351198,0.343430,0.335514,0.327328,0.318752,0.309922,0.301990,0.296299,0.292733,...,0.285920,0.281396,0.276505,0.271679,0.267411,0.264026,0.261159,0.258273,0.254831,0.250296
-16.5,0.357093,0.349338,0.341578,0.333691,0.325554,0.317046,0.308294,0.300423,0.294746,0.291149,...,0.284287,0.279779,0.274925,0.270143,0.265909,0.262532,0.259662,0.256782,0.253373,0.248920
-16.0,0.355229,0.347463,0.339711,0.331852,0.323763,0.315322,0.306651,0.298846,0.293183,0.289553,...,0.282645,0.278157,0.273341,0.268604,0.264402,0.261035,0.258164,0.255290,0.251916,0.247542
-15.5,0.353343,0.345571,0.337829,0.329997,0.321954,0.313581,0.304995,0.297258,0.291611,0.287947,...,0.280995,0.276531,0.271756,0.267061,0.262890,0.259533,0.256663,0.253798,0.250458,0.246161


In [90]:
pd.DataFrame(implied_vols, index=[temp_data2['T'], temp_data2['K']])

T      K     
0.250  80   0    0.369550
0.375  80   0    0.350208
0.500  80   0    0.328635
0.625  80   0    0.307913
0.750  80   0    0.299198
0.875  80   0    0.288520
1.000  80   0    0.276329
1.125  80   0    0.268711
1.250  80   0    0.258496
0.250  85   0    0.351437
0.375  85   0    0.332045
0.500  85   0    0.311824
0.625  85   0    0.292533
0.750  85   0    0.283154
0.875  85   0    0.272550
1.000  85   0    0.261374
1.125  85   0    0.253761
1.250  85   0    0.244778
0.250  90   0    0.331549
0.375  90   0    0.312468
0.500  90   0    0.293518
0.625  90   0    0.276143
0.750  90   0    0.266094
0.875  90   0    0.256276
1.000  90   0    0.246061
1.125  90   0    0.238787
1.250  90   0    0.230846
0.250  95   0    0.311607
0.375  95   0    0.292254
0.500  95   0    0.274908
                   ...   
1.000  105  0    0.200403
1.125  105  0    0.195300
1.250  105  0    0.189809
0.250  110  0    0.245132
0.375  110  0    0.228185
0.500  110  0    0.215678
0.625  110  0    0.20388

In [64]:
temp_data['price'].values.min()

1.92558393057222

In [65]:
z = all_optimize(temp_data)

In [72]:
(temp_data['T'].values, temp_data['K'].values)

(array([0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ,
        0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 ]),
 array([ 80,  80,  80,  80,  80,  80,  80,  80,  80,  85,  85,  85,  85,
         85,  85,  85,  85,  85,  90,  90,  90,  90,  90,  90,  90,  90,
         90,  95,  95,  95,  95,  95,  95,  95,  95,  95, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 105, 105, 105, 105, 105, 105, 105,
        105, 105, 110, 110, 110, 110, 110, 110, 110, 110, 110, 115,

In [71]:
x = pd.DataFrame(z, index=(temp_data['T'].values, temp_data['K'].values))

ValueError: Shape of passed values is (81, 1), indices imply (2, 1)

In [69]:
x.index = [x['T'], x['K']]

In [70]:
x

,,T,K,implied_vol
T,K,,,
0.250,80,0.250,80,0.080097
0.375,80,0.375,80,0.069686
0.500,80,0.500,80,0.060564
0.625,80,0.625,80,0.054541
0.750,80,0.750,80,0.026577
0.875,80,0.875,80,0.011800
1.000,80,1.000,80,0.002177
1.125,80,1.125,80,0.034120
1.250,80,1.250,80,0.051277


In [26]:
dir(o1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_calculate_statistics',
 '_repr_html_',
 'aborted',
 'aic',
 'bic',
 'chisqr',
 'errorbars',
 'flatchain',
 'fun',
 'hess_inv',
 'init_vals',
 'init_values',
 'jac',
 'message',
 'method',
 'ndata',
 'nfev',
 'nfree',
 'nit',
 'njev',
 'nvarys',
 'params',
 'redchi',
 'residual',
 'show_candidates',
 'status',
 'success',
 'var_names',
 'x']

In [53]:
o1.params.valuesdict()['sigma']

0.33644620572351147